# IMPORT PACKAGE

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn import svm
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# IMPORT DATA

In [2]:
# Load the dataset
trainingSet = pd.read_csv("./data/X_train.csv")
submissionSet = pd.read_csv("./data/X_test.csv")

# DATA PROCESSING

In [3]:
def process(trainingSet,submissionSet,col = 'Text'):
    training_helpful = trainingSet[(trainingSet['HelpfulnessNumerator']<=trainingSet['HelpfulnessDenominator'])]
    training_drop = training_helpful.dropna()
    print("train set after cleaning wrong in helpfulness:   " , trainingSet.shape)
    print("train set after drop NaN:   ",training_drop.shape)

    X_train, X_test, Y_train, Y_test = train_test_split(
        training_drop.drop(['Score'], axis=1),
        training_drop['Score'],
        test_size=1/4.0,
        random_state=0
    )


    if col == 'Text':
        drop_col = ['Id', 'ProductId', 'UserId', 'Summary', 'Time']
    elif col == 'Summary':
        drop_col = ['Id', 'ProductId', 'UserId', 'Text', 'Time']

    X_train_processed = X_train.drop(columns = drop_col)
    X_test_processed = X_test.drop(columns = drop_col)
    submission_processed = submissionSet.drop(columns = drop_col)
    print("train set shape:  ",X_train_processed.shape,"test set shape:  ",X_test_processed.shape)
    
    return X_train_processed,X_test_processed,Y_train,Y_test,submission_processed

In [4]:
#clean the text
#https://stackoverflow.com/questions/5843518/remove-all-special-characters-punctuation-and-spaces-from-string
#https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe
def remove_char(x):
    special = '[^A-Za-z ]+'
    x = re.sub(special,'',x)
    x = x.strip()
    x = x.lower()
    return x


def clean_word(dataset,col):
    stop_words = set(stopwords.words('english'))
    test = dataset[col].apply(lambda row: remove_char(str(row))).apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return test

In [5]:
trainingSet['Text'] = trainingSet['Text'] + ' ' + trainingSet['Summary']
submissionSet['Text'] = submissionSet['Text']  + ' ' + submissionSet['Summary']

In [6]:
#SEPERATE THE DATASET 
X_train_processed,X_test_processed,Y_train,Y_test,submission_processed = process(trainingSet,submissionSet,'Text')

train set after cleaning wrong in helpfulness:    (1397533, 10)
train set after drop NaN:    (1397455, 10)
train set shape:   (1048091, 4) test set shape:   (349364, 4)


In [7]:
X_train_processed.head()

,HelpfulnessNumerator,HelpfulnessDenominator,Text,Helpfulness
214099,0,0,"As a big fan of Miami Vice, I enjoyed this mov...",0.0
544327,0,0,Intimate and fascinating biographic documentar...,0.0
500542,1,2,This movie was really not all that great. The...,0.5
493726,0,0,Tried watching this on TV but missed several e...,0.0
579893,0,0,it's ok if you had last few movies on the she...,0.0


In [8]:
# APLLY THE CLEANNING FUNCTION TO DATAFRAME
X_train_text = clean_word(X_train_processed,'Text')
X_test_text = clean_word(X_test_processed,'Text')
submission_text = clean_word(submission_processed,'Text')
X_train_text.head()

214099    big fan miami vice enjoyed movie mainly enjoy ...
544327    intimate fascinating biographic documentary gr...
500542    movie really great way movie designed looked l...
493726    tried watching tv missed several episodes enjo...
579893    ok last movies shelf would reccomend someone t...
Name: Text, dtype: object

In [9]:
#countVector the data
vectorizer_text = CountVectorizer()
X_training_vector_text = vectorizer_text.fit_transform(X_train_text)
X_test_vectorr_text = vectorizer_text.transform(X_test_text)
submission_vectorr_text = vectorizer_text.transform(submission_text)
print("shape of training set CountVector&text: ",X_training_vector_text.shape)

shape of training set CountVector&text:  (1048091, 1755702)


In [9]:
#tfidf the data
tfidf_text = TfidfVectorizer()
X_training_tfidf_text = tfidf_text.fit_transform(X_train_text)
X_test_tfidf_text = tfidf_text.transform(X_test_text)
submission_tfidf_text = tfidf_text.transform(submission_text)
print("shape of training set TFIDF&text: ",X_training_tfidf_text.shape)

shape of training set TFIDF&text:  (1048091, 1755702)


In [11]:


clf = LogisticRegression(random_state=0).fit(X_training_vector_text,Y_train)
logprevec = clf.predict(X_test_vectorr_text)
print("RMSE on 1 testing set = ", mean_squared_error(Y_test, logprevec))

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE on 1 testing set =  0.8971531125130237


In [12]:

clf2 = LogisticRegression(random_state=0).fit(X_training_tfidf_text,Y_train)
logprevec2 = clf2.predict(X_test_tfidf_text)
print("RMSE on 2 testing set = ", mean_squared_error(Y_test, logprevec2))

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE on 2 testing set =  0.8653438820256237


In [17]:
clf3 = LogisticRegression(random_state=0,C=20).fit(X_training_tfidf_text,Y_train)
logprevec3 = clf3.predict(X_test_tfidf_text)
print("RMSE on testing set = ", mean_squared_error(Y_test, logprevec3))

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE on 2 testing set =  0.862676177282147


In [18]:
clf4 = LogisticRegression(random_state=0,C=40).fit(X_training_tfidf_text,Y_train)
logprevec4 = clf4.predict(X_test_tfidf_text)
print("RMSE on testing set = ", mean_squared_error(Y_test, logprevec4))

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE on 2 testing set =  0.8577558076962709


In [19]:
clf5 = LogisticRegression(random_state=0,C=60).fit(X_training_tfidf_text,Y_train)
logprevec5 = clf5.predict(X_test_tfidf_text)
print("RMSE on testing set = ", mean_squared_error(Y_test, logprevec5))

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE on 2 testing set =  0.8518135812505009


In [21]:
clf6 = LogisticRegression(random_state=0,C=100).fit(X_training_tfidf_text,Y_train)
logprevec6 = clf6.predict(X_test_tfidf_text)
print("RMSE on testing set = ", mean_squared_error(Y_test, logprevec6))

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE on testing set =  0.9128845559359293


In [23]:
clf7 = LogisticRegression(random_state=0,C=80).fit(X_training_tfidf_text,Y_train)
logprevec7 = clf7.predict(X_test_tfidf_text)
print("RMSE on testing set = ", mean_squared_error(Y_test, logprevec7))

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE on 2 testing set =  0.8435757548001511


In [25]:
logprevec8 = LogisticRegression(random_state=0,C=90).fit(X_training_tfidf_text,Y_train).predict(X_test_tfidf_text)
print("RMSE on testing set = ", mean_squared_error(Y_test, logprevec8))

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE on testing set =  0.8593472710410918


In [12]:
lg8 = LogisticRegression(random_state=0,C=80,max_iter=100).fit(X_training_tfidf_text,Y_train)

C:\Users\dsa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
logprevec8 = lg8.predict(X_test_tfidf_text)
print("RMSE on testing set = ", mean_squared_error(Y_test, logprevec8))

RMSE on testing set =  0.8302572674917851


In [14]:
lg8 = LogisticRegression(random_state=0,C=80,max_iter=1000).fit(X_training_tfidf_text,Y_train)

In [ ]:
logprevec9 = LogisticRegression(random_state=0,C=80,max_iter=5000).fit(X_training_tfidf_text,Y_train).predict(X_test_vectorr_text)
print("RMSE on testing set = ", mean_squared_error(Y_test, logprevec9))

In [26]:
submission_predict = submissionSet
submission_predict['Score'] = clf7.predict(submission_tfidf_text)
submission_output = submission_predict[['Id','Score']]
submission_output.to_csv("./data/lg7_tfidf.csv",index = False)